In [2]:
import numpy as np
import scipy as sci
from scipy.optimize import curve_fit
from scipy.special import voigt_profile
import matplotlib.pyplot as plt 
import pandas as pd

**Actividad 1**

En esta actividad la idea principal es medir el espectro característico de rayos X del cobre (Cu). Para esto, se utilizó el montaje  que se ve en la figura. Tomamos datos como se indicó en la guía. Acá primero vamos a definir las funciones de ajuste que usaremos en el análisis.


In [3]:
def Lorentz(x, A, x0, gamma):
    return (A / np.pi) * (gamma / ((x - x0)**2 + gamma**2))


def Linear(x, a, b):
    return a*x + b
